# KoBigBird_Keyword_Extraction_Frequancy_Weighted

단어의 빈도수에 따른 가중치와 DBSCAN을 사용한 동의어제거를 적용한 최종 코드

## 본문 키워드 형태소 분석하여 후보 키워드 추출하기

형태소 분석기 komoran을 사용하기 위해 konlpy 설치

In [ ]:
#!pip install konlpy

### 불용어 제거

In [3]:
from nltk.corpus import stopwords as eng_stopwords

eng_stopwords_set = set(eng_stopwords.words('english'))

In [4]:
def remove_stopwords(nouns_list, stopword_set):
    arr_removed = [n for n in nouns_list if n not in stopword_set]
    return arr_removed

In [5]:
import csv

stopwords = []
with open('./data/stopwords/stopwords.txt', 'r') as f:
    for line in f:
        stopwords.append(line.rstrip('\n'))

with open('./data/stopwords/stopwords-ko.txt', 'r') as f:
    for line in f:
        stopwords.append(line.rstrip('\n'))
        
with open('./data/stopwords/stopwords-np.txt', 'r') as f:
    tsv_file = csv.reader(f, delimiter="\t")
    for line in tsv_file:
        stopwords.append(line[0])

stopwords_set = set(stopwords)
print(stopwords_set)

{'고로', '딩동', '아니면', '？', '다시 말하자면', '‘', '나오', '어느 년도', '한 까닭에', '알 수 있다', '할만하다', '이럴정도로', '하곤하였다', '주저하지 않고', '로 인하여', '우리', '바로', '혹시', '뤼', '하자마자', '바꾸어서 한다면', '얼마간', '저기', '버', '대해 말하자면', '경우', '까악', '잘', '어떤', '의', '이 때문에', '그중에서', '마저도', '이래', '요만한걸', '줄은 몰랏다', '두', '좍좍', '하고있었다', '하마터면', '심지어', '하는것이 낫다', '주', '받', '여섯', '조차', '그리하여', '할지언정', '너', '즈음하여', '드', '야', '데', '연이서', '입각하여', '정도에 이르다', '실로', '생각한대로', '반드시', '이러이러하다', '이 되다', '오자마자', '텐', '하는 김에', '이것', '생각하', '이와같다면', '＃', '어떠한', '하기만 하면', '어찌됏어', '않기 위해서', '너희들', '어느때', '에 한하다', '아야', '소인', '앞에서', '＋', '때가 되어', '할수있다', '어때', '～', '허걱', '하하', '하', '무릎쓰고', '하지 않도록', '또', '아이', '에', '하든지', '그래', '해봐요', '매번', '설사', '영차', '오호', '하는 편이 낫다', '근거하여', '우리들', '위하여', '하더라도', '각각', '자마자', '시각', '그것', '거바', '어디', '아니라면', '만일', '대하면', '겨우', '힘입어', '그렇', '다른', '하면 할수록', '가지', '무렵', '툭', '뒤따라', '휴', '공동으로', '그럼에도 불구하고', '따라서', '때', '까지 미치다', '마음대로', '］', '지만', '두번째로', '인젠', '있', '총적으로', '(', '하지 않는다면', '등', '구토하다', '관한', '저것', '구체적으

### 명사구 추출 함수 정의

In [6]:
import konlpy
import nltk
from konlpy.tag import Okt

def get_nouns_phrase(text):  
    okt = Okt()
    phrases = okt.phrases(text)
    return phrases

In [7]:
# from konlpy.tag import Mecab

# mecab = Mecab()

# def extract_noun_phrases(text):
#     tagged_tokens = mecab.pos(text)
#     noun_phrases = []
#     current_noun_phrase = []
#     tokens = [token for token, _ in tagged_tokens]

#     for i, (word, pos) in enumerate(tagged_tokens):
#         if pos.startswith('N') or pos.startswith('MM') or (pos.startswith('VA') and i < len(tagged_tokens) - 1 and tagged_tokens[i + 1][1].startswith('N')) or pos.startswith('XR'):
#             current_noun_phrase.append((i, pos))
#         else:
#             if current_noun_phrase:
#                 start_idx = current_noun_phrase[0][0]
#                 end_idx = current_noun_phrase[-1][0]
#                 noun_phrase = ' '.join(tokens[start_idx:end_idx + 1])

#                 if len(noun_phrase.split()) > 1 or len(noun_phrase) > 1:
#                     noun_phrases.append(noun_phrase)

#                 current_noun_phrase = []

#         if i < len(tagged_tokens) - 1 and pos.startswith('N') and not tagged_tokens[i + 1][1].startswith('N'):
#             if current_noun_phrase:
#                 start_idx = current_noun_phrase[0][0]
#                 end_idx = current_noun_phrase[-1][0]
#                 noun_phrase = ' '.join(tokens[start_idx:end_idx + 1])

#                 if len(noun_phrase.split()) > 1 or len(noun_phrase) > 1:
#                     noun_phrases.append(noun_phrase)

#                 current_noun_phrase = []

#     if current_noun_phrase:
#         start_idx = current_noun_phrase[0][0]
#         end_idx = current_noun_phrase[-1][0]
#         noun_phrase = ' '.join(tokens[start_idx:end_idx + 1])

#         if len(noun_phrase.split()) > 1 or len(noun_phrase) > 1:
#             noun_phrases.append(noun_phrase)

#     return noun_phrases

### 각 형태소 분석기별 명사 추출 함수 정의

형태소 분석을 통해 명사 추출, 불용어 제거, 영어 단어 추출하는 함수

In [8]:
from konlpy.tag import Mecab 
from konlpy.tag import Komoran
from konlpy.tag import Kkma
from konlpy.tag import Okt
from collections import Counter

def get_nouns_mecab(text, use_phrase=False):
    mecab = Mecab()
    nouns = mecab.nouns(text)
    if use_phrase:
        phrases = get_nouns_phrase(text)
        nouns.extend(phrases)
    eng_nouns = get_nouns_eng(text)
    eng_nouns = [ n for n in eng_nouns if len(n) != 1 ]
    eng_nouns = remove_stopwords(eng_nouns, eng_stopwords_set)
    nouns.extend(eng_nouns)
    nouns = remove_stopwords(nouns, stopwords_set)
    counts = Counter(nouns)
    nouns = set(nouns)
    nouns = list(nouns)
    
    return nouns, counts

def get_nouns_komoran(text, use_phrase=False):
    komoran = Komoran()
    nouns = komoran.nouns(text)
    if use_phrase:
        phrases = get_nouns_phrase(text)
        nouns.extend(phrases)
    eng_nouns = get_nouns_eng(text)
    eng_nouns = [ n for n in eng_nouns if len(n) != 1 ]
    eng_nouns = remove_stopwords(eng_nouns, eng_stopwords_set)
    nouns.extend(eng_nouns)
    nouns = remove_stopwords(nouns, stopwords_set)
    counts = Counter(nouns)
    nouns = set(nouns)
    nouns = list(nouns)
    
    return nouns, counts

def get_nouns_kkma(text):
    kkma = Kkma()
    nouns = kkma.nouns(text)
    nuons = remove_stopwords(nouns, stopwords_set)
    nouns = set(nouns)
    nouns = list(nouns)
    eng_nouns = get_nouns_eng(text)
    eng_nouns = [ n for n in eng_nouns if len(n) != 1 ]
    nouns.extend(eng_nouns)
    return nouns

def get_nouns_okt(text):
    okt = Okt()
    nouns = okt.nouns(text)
    nuons = remove_stopwords(nouns, stopwords_set)
    nouns = set(nouns)
    nouns = list(nouns)
    eng_nouns = get_nouns_eng(text)
    eng_nouns = [ n for n in eng_nouns if len(n) != 1 ]
    nouns.extend(eng_nouns)
    
    return nouns

In [9]:
import nltk

from string import punctuation
import re

def get_nouns_eng(text):
    # 명사 추출 시 url 제거
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    text = re.sub(url_pattern, '', text)
    url_pattern = re.compile(r"[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{2,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)")
    text = re.sub(url_pattern, '', text)
    
    text = text.encode('utf-8').decode('ascii', 'ignore')

    result = ""
    for t in text:
        if t not in punctuation:
            result += t

    text = result.strip()

    word_tokens = nltk.word_tokenize(text)
    tokens_pos = nltk.pos_tag(word_tokens)
    NN_words = []
    for word, pos in tokens_pos:
        if 'NN' in pos or 'NNP' in pos:
            NN_words.append(word)
            
    return NN_words

### 데이터 불러오기

In [10]:
import jsonlines

sample = []

with jsonlines.open("./data/brunch_NLI_test.jsonl") as f:
    for line in f.iter():
        if line['likes'] >= 180:
            sample.append(line)

In [11]:
len(sample)

217

### 각 KoBigBird 모델 불러오기

In [12]:
from transformers import  AutoModel,AutoTokenizer

model = AutoModel.from_pretrained("monologg/kobigbird-bert-base", attention_type="original_full", add_pooling_layer=False)
#model = AutoModel.from_pretrained("./model/sts/saved_model_epoch_7.pt", attention_type="original_full", add_pooling_layer=False)
#model = AutoModel.from_pretrained("./model/nli/saved_model_epoch_2.pt", attention_type="original_full", add_pooling_layer=False)
#model = AutoModel.from_pretrained("./model/word_to_title/saved_model_epoch_3.pt", attention_type="original_full", add_pooling_layer=False)
#model = AutoModel.from_pretrained("./model/title_to_text/saved_model_epoch_2.pt", attention_type="original_full", add_pooling_layer=False)
#model = AutoModel.from_pretrained("./model/word_to_text_fm3/saved_model_epoch_3.pt", attention_type="original_full", add_pooling_layer=False)
#model = AutoModel.from_pretrained("./model/word_to_text_sts_way/saved_model_epoch_2.pt", attention_type="original_full", add_pooling_layer=False)

tokenizer = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base")

/home/user03/anaconda3/envs/mecab/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at monologg/kobigbird-bert-base were not used when initializing BigBirdModel: ['cls.predictions.decoder.weight', 'bert.pooler.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'bert.pooler.bias']
- This IS expected if you are initializing BigBirdModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected

In [13]:
import torch
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"

model.to('cuda')
model.eval()

BigBirdModel(
  (embeddings): BigBirdEmbeddings(
    (word_embeddings): Embedding(32500, 768, padding_idx=0)
    (position_embeddings): Embedding(4096, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BigBirdEncoder(
    (layer): ModuleList(
      (0): BigBirdLayer(
        (attention): BigBirdAttention(
          (self): BigBirdSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BigBirdSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inp

## 최종 키워드 추출 함수(빈도수에 따른 가중치 반영)

In [14]:
def extract_main_keywords(nouns, counts, title, text, model, tokenizer, use_freq_w=False):
    
    tokenized_nouns = {}
    for keyword in nouns:
        tokenized_nouns[keyword] = tokenizer(keyword, max_length=32, padding='max_length', return_tensors='pt')
    
    keyword_vectors = {}
    for keyword, tokenized in tokenized_nouns.items():
        output = model(**tokenized.to(device))[0][:,0,:]
        keyword_vectors[keyword] = output
        
    tokenized_title = tokenizer(title, max_length=128, padding="max_length", return_tensors='pt')
    tokenized_text = tokenizer(text, max_length=4096, padding="max_length", return_tensors='pt')

    title_vec = model(**tokenized_title.to(device))[0][:,0,:]
    body_text_vec = model(**tokenized_text.to(device))[0][:,0,:]
    
    
    if use_freq_w:
        total_words_counts = 0
        for word, count in counts.items():
            if count != 1:
                total_words_counts += count
        print(total_words_counts)
        
        freq_w = {}
    
        for keyword in nouns:
            if counts[keyword] <= 1:
                freq_w[keyword] = 0
                continue
            freq_w[keyword] = (counts[keyword]/total_words_counts)
        
        print(freq_w)
        for keyword, vec in keyword_vectors.items():
            body_text_vec = ((freq_w[keyword]* vec) + body_text_vec)
        
        body_text_vec = body_text_vec/total_words_counts
        
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    similarity = {}

    for keyword, vector in keyword_vectors.items():
        sim = cos(vector,  title_vec*(0.0) + body_text_vec*(1.0))
        similarity[keyword] = sim
        
    sorted_dict = sorted(similarity.items(), key = lambda item: item[1], reverse = True)
    
    # print(keyword_vectors)
    print(f"제목과 본문의 유사도 {cos(title_vec, body_text_vec)}\n")
    print(title+'\n')
    print(text+'\n')
    print(sorted_dict,'\n')
    #print(nouns)
    
    return keyword_vectors, similarity

## 클러스터링 후 클러스터 내 1위 단어만 추출(동의어 제거)

사이킷런의 DBSCAN 사용

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
  
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA


def cluster_dbscan(eps, min_samples, vec_dict, sim_dict, max_words=10):
    word_features_numpy = {}
    for word, feat in vec_dict.items():
        word_features_numpy[word] = feat.to('cpu')[0].numpy()
        
    features_df = pd.DataFrame(word_features_numpy)
    features_df = features_df.transpose()
    
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(features_df)
    DBSCAN_dataset = features_df.copy()
    DBSCAN_dataset.loc[:,'Cluster'] = clustering.labels_ 
    
    print(DBSCAN_dataset.Cluster.value_counts().to_frame())
    print(clustering.labels_ .max())
    
    sim_list = []
    for word in DBSCAN_dataset.index:
        sim_list.append(sim_dict[word].item())
        
    DBSCAN_dataset['sim'] = sim_list
    
    cluster_list = DBSCAN_dataset.Cluster.to_list()
    cluseter_set = set(cluster_list)
    cluster_list = list(cluseter_set)
    
    selected_words = []

    for cluster in cluster_list:
        if cluster == -1 or cluster == 0:
            df = DBSCAN_dataset[DBSCAN_dataset['Cluster']==cluster]
            #print(df.index.to_list())
            selected_words.extend(df.index.to_list())
        else:
            df = DBSCAN_dataset[DBSCAN_dataset['Cluster']==cluster]
            df = df.sort_values('sim', ascending=False)
            top_word = df.iloc[0].name
            selected_words.append(top_word)
            
    selected_words_dict = {}

    for word in selected_words:
        selected_words_dict[word] = sim_dict[word]
    sorted_selected_words = sorted(selected_words_dict.items(), key = lambda item: item[1], reverse = True)
    
    return sorted_selected_words[:max_words]

In [16]:
sample_essay = sample[200]

with torch.no_grad():
    print(sample_essay['keywords'])
    
    title = sample_essay['title']
    text = sample_essay['body_text']
    text = text.replace('\n','').replace('\t','').replace('\r','') # 코모란 에러 방지
    nouns, counts = get_nouns_mecab(text, use_phrase=False)
    vec_dict, sim_dict = extract_main_keywords(nouns, counts, title, text, model, tokenizer, use_freq_w=False)
    keywords = cluster_dbscan(eps=4.9, min_samples=2, vec_dict=vec_dict, sim_dict=sim_dict, max_words=20)
    print(keywords)

['가족', '에세이', '호칭']
제목과 본문의 유사도 tensor([0.9065], device='cuda:0')

나는 가끔 아파를 아빠라 부른다.

 내 글을 우연히 읽었다며 지인은 물었다.  매영씨, 매영씨는 왜 아버지를 아빠라 부르시나요? 아버지가 더 거리감 있어 보이는데요. 아빠라는 호칭은 너무 가깝지 않나요? 어색한 웃음이 흘러나왔다. 나는 왜 아빠를 아빠라 부르는 걸까. 마땅한 대답이 생각나지 않았다. 날이 좋다고, 오늘은 우리 맛있는 것을 먹자며 말을 돌렸다. 창 밖에는 비가 오고 있었다. 헤어지고 집에 오는 길에 아버지라는 말을 내내 중얼거렸다. 기름칠을 한 것처럼 입에서 미끄러져 흘렀다. 아버지라는 말을 어디서 처음 들었던가. 드라마였던 것 같다. 드라마 속 아버지들은 대부분 권위적이지만 능력이 있었다. 아들에게 좋은 스승이자 라이벌이었다. 아들과 아버지가 싸우는 이유는 뛰어넘기 위한 발버둥이거나 사랑 때문이었다. 우리에겐 없는 것이었다. 집에 도착해서는 아빠라는 말을 한참 동안 중얼거렸다. 아빠. 아파. 아파. 아빠 발음할수록 아빠는 아파가 되고 아파는 아빠가 되었다. 아직도 폭력의 여운이 사라지지 않았다는 것이겠지. 내게 아버지란 단어는 너무 멀고 무겁다.   어떻게 아빠를 아빠라고 부르지 않을 수 있을까. 

[('아빠', tensor([0.9181], device='cuda:0')), ('아버지', tensor([0.9043], device='cuda:0')), ('대답', tensor([0.9012], device='cuda:0')), ('사랑', tensor([0.8968], device='cuda:0')), ('스승', tensor([0.8966], device='cuda:0')), ('길', tensor([0.8926], device='cuda:0')), ('창', tensor([0.8917], device='cuda:0')), ('글', tensor([0.8904], device='cuda:0')), ('발버둥', tens

### kr-wordrank와 비교

In [17]:
from krwordrank.word import KRWordRank

min_count = 1   # 단어의 최소 출현 빈도수 (그래프 생성 시)
max_length = 15 # 단어의 최대 길이
wordrank_extractor = KRWordRank(min_count=min_count, max_length=max_length)

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 20
texts = [text]
keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)

for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:20]:
        print('%8s:\t%.4f' % (word, r))

      아빠:	5.2586
      말을:	4.1436
      아버:	2.9174
      사랑:	2.8845
      집에:	2.8730
      위한:	2.5356
     대부분:	2.5059
    있었다.:	2.3040
      있어:	2.1997
     능력이:	2.1779
     웃음이:	1.9623
     아들과:	1.9526
      너무:	1.9155
    않았다는:	1.9000
      없는:	1.8874
    스승이자:	1.8707
      처음:	1.8506
  중얼거렸다.:	1.8174
      좋은:	1.7735
     지인은:	1.7099
